### All days of the challange:

Here's what we're going to do today:

* [Take a first look at the data](#Take-a-first-look-at-the-data)
* [See how many missing data points we have](#See-how-many-missing-data-points-we-have)
* [Figure out why the data is missing](#Figure-out-why-the-data-is-missing)
* [Drop missing values](#Drop-missing-values)
* [Filling in missing values](#Filling-in-missing-values)

Let's get started!

# Take a first look at the data
________

The first thing we'll need to do is load in the libraries and datasets we'll be using. For today, I'll be using a dataset of events that occured in American Football games for demonstration, and you'll be using a dataset of building permits issued in San Francisco.

> **Important!** Make sure you run this cell yourself or the rest of your code won't work!

In [7]:
import pandas as pd
nfl1 = pd.read_csv("NFL Play by Play 2009-2017 (v4).csv")
nfl2 = pd.read_csv("NFL Play by Play 2009-2016 (v3).csv")






<ipython-input-7-0f634fab97a3>:2: DtypeWarning: Columns (51) have mixed types. Specify dtype option on import or set low_memory=False.
  nfl1 = pd.read_csv("NFL Play by Play 2009-2017 (v4).csv")
<ipython-input-7-0f634fab97a3>:3: DtypeWarning: Columns (51) have mixed types. Specify dtype option on import or set low_memory=False.
  nfl2 = pd.read_csv("NFL Play by Play 2009-2016 (v3).csv")


The first thing I do when I get a new dataset is take a look at some of it. This lets me see that it all read in correctly and get an idea of what's going on with the data. In this case, I'm looking to see if I see any missing values, which will be reprsented with `NaN` or `None`.

In [8]:
print(nfl1.head())


         Date      GameID  Drive  qtr  down   time  TimeUnder  TimeSecs  \
0  2009-09-10  2009091000      1    1   NaN  15:00         15    3600.0   
1  2009-09-10  2009091000      1    1   1.0  14:53         15    3593.0   
2  2009-09-10  2009091000      1    1   2.0  14:16         15    3556.0   
3  2009-09-10  2009091000      1    1   3.0  13:35         14    3515.0   
4  2009-09-10  2009091000      1    1   4.0  13:27         14    3507.0   

   PlayTimeDiff SideofField  ...    yacEPA  Home_WP_pre  Away_WP_pre  \
0           0.0         TEN  ...       NaN     0.485675     0.514325   
1           7.0         PIT  ...  1.146076     0.546433     0.453567   
2          37.0         PIT  ...       NaN     0.551088     0.448912   
3          41.0         PIT  ... -5.031425     0.510793     0.489207   
4           8.0         PIT  ...       NaN     0.461217     0.538783   

   Home_WP_post  Away_WP_post  Win_Prob       WPA    airWPA    yacWPA  Season  
0      0.546433      0.453567  0.485

Yep, it looks like there's some missing values. What about in the nfl2 dataset?

In [9]:
# your turn! Look at a couple of rows from the nfl2 dataset. Do you notice any missing data?

print(nfl2.head())


         Date      GameID  Drive  qtr  down   time  TimeUnder  TimeSecs  \
0  2009-09-10  2009091000      1    1   NaN  15:00         15    3600.0   
1  2009-09-10  2009091000      1    1   1.0  14:53         15    3593.0   
2  2009-09-10  2009091000      1    1   2.0  14:16         15    3556.0   
3  2009-09-10  2009091000      1    1   3.0  13:35         14    3515.0   
4  2009-09-10  2009091000      1    1   4.0  13:27         14    3507.0   

   PlayTimeDiff SideofField  ...    yacEPA  Home_WP_pre  Away_WP_pre  \
0           0.0         TEN  ...       NaN     0.485675     0.514325   
1           7.0         PIT  ...  1.146076     0.546433     0.453567   
2          37.0         PIT  ...       NaN     0.551088     0.448912   
3          41.0         PIT  ... -5.031425     0.510793     0.489207   
4           8.0         PIT  ...       NaN     0.461217     0.538783   

   Home_WP_post  Away_WP_post  Win_Prob       WPA    airWPA    yacWPA  Season  
0      0.546433      0.453567  0.485

# See how many missing data points we have
___

Ok, now we know that we do have some missing values. Let's see how many we have in each column.

In [10]:
missing_data_counts = nfl1.isnull().sum()
print("Number of missing points in the first ten columns:")
print(missing_data_counts[:10])
missing_data_counts = nfl2.isnull().sum()
print("Number of missing points in the first ten columns:")
print(missing_data_counts[:10])


Number of missing points in the first ten columns:
Date               0
GameID             0
Drive              0
qtr                0
down            2385
time              12
TimeUnder          0
TimeSecs          12
PlayTimeDiff      24
SideofField       22
dtype: int64
Number of missing points in the first ten columns:
Date               0
GameID             0
Drive              0
qtr                0
down            3120
time              17
TimeUnder          0
TimeSecs          17
PlayTimeDiff      34
SideofField       29
dtype: int64


That seems like a lot! It might be helpful to see what percentage of the values in our dataset were missing to give us a better sense of the scale of this problem:

In [14]:
total_missing_values = nfl1.isnull().sum().sum()
total_entries = nfl1.shape[0] * nfl1.shape[1]
percent_missing = (total_missing_values / total_entries) * 100

print("Total number of missing values in nfl1:", total_missing_values)
print("Percentage of data that is missing: {:.3f}%".format(percent_missing))




Total number of missing values in nfl1: 441471
Percentage of data that is missing: 27.777%


Wow, almost a quarter of the cells in this dataset are empty! In the next step, we're going to take a closer look at some of the columns with missing values and try to figure out what might be going on with them.

In [15]:
total_missing_values = nfl2.isnull().sum().sum()
total_entries = nfl2.shape[0] * nfl2.shape[1]
percent_missing = (total_missing_values / total_entries) * 100

print("Total number of missing values in nfl2:", total_missing_values)
print("Percentage of data that is missing: {:.3f}%".format(percent_missing))


Total number of missing values in nfl2: 574638
Percentage of data that is missing: 27.804%


# Figure out why the data is missing
____

This is the point at which we get into the part of data science that I like to call "data intution", by which I mean "really looking at your data and trying to figure out why it is the way it is and how that will affect your analysis". It can be a frustrating part of data science, especially if you're newer to the field and don't have a lot of experience. For dealing with missing values, you'll need to use your intution to figure out why the value is missing. One of the most important question you can ask yourself to help figure this out is this:

> **Is this value missing becuase it wasn't recorded or becuase it dosen't exist?**

If a value is missing becuase it doens't exist (like the height of the oldest child of someone who doesn't have any children) then it doesn't make sense to try and guess what it might be. These values you probalby do want to keep as NaN. On the other hand, if a value is missing becuase it wasn't recorded, then you can try to guess what it might have been based on the other values in that column and row. (This is called "imputation" and we'll learn how to do it next! :)

Let's work through an example. Looking at the number of missing values in the nfl_data dataframe, I notice that the column `TimesSec` has a lot of missing values in it:

In [16]:
missing_data = nfl1.iloc[:, :10].isnull().sum()
print("Number of missing points in the first ten columns:")
print(missing_data)



Number of missing points in the first ten columns:
Date               0
GameID             0
Drive              0
qtr                0
down            2385
time              12
TimeUnder          0
TimeSecs          12
PlayTimeDiff      24
SideofField       22
dtype: int64


By looking at [the documentation](https://www.kaggle.com/maxhorowitz/nflplaybyplay2009to2016), I can see that this column has information on the number of seconds left in the game when the play was made. This means that these values are probably missing because they were not recorded, rather than because they don't exist. So, it would make sense for us to try and guess what they should be rather than just leaving them as NA's.

On the other hand, there are other fields, like `PenalizedTeam` that also have lot of missing fields. In this case, though, the field is missing because if there was no penalty then it doesn't make sense to say *which* team was penalized. For this column, it would make more sense to either leave it empty or to add a third value like "neither" and use that to replace the NA's.

> **Tip:** This is a great place to read over the dataset documentation if you haven't already! If you're working with a dataset that you've gotten from another person, you can also try reaching out to them to get more information.

If you're doing very careful data analysis, this is the point at which you'd look at each column individually to figure out the best strategy for filling those missing values. For the rest of this notebook, we'll cover some "quick and dirty" techniques that can help you with missing values but will probably also end up removing some useful information or adding some noise to your data.

## Your turn!

* Look at the columns `Street Number Suffix` and `Zipcode` from the `sf_permits` datasets. Both of these contain missing values. Which, if either, of these are missing because they don't exist? Which, if either, are missing because they weren't recorded?

# Drop missing values
___

If you're in a hurry or don't have a reason to figure out why your values are missing, one option you have is to just remove any rows or columns that contain missing values. (Note: I don't generally recommend this approch for important projects! It's usually worth it to take the time to go through your data and really look at all the columns with missing values one-by-one to really get to know your dataset.)  

If you're sure you want to drop rows with missing values, pandas does have a handy function, `dropna()` to help you do this. Let's try it out on our NFL dataset!

In [17]:
nfl1_cleaned = nfl1.dropna()
print("DataFrame after removing rows with missing values:", nfl1_cleaned.shape)


DataFrame after removing rows with missing values: (0, 102)


Oh dear, it looks like that's removed all our data! 😱 This is because every row in our dataset had at least one missing value. We might have better luck removing all the *columns* that have at least one missing value instead.

In [18]:
nfl1_cleaned = nfl1.dropna(axis=1)
print("Shape of DataFrame after removing columns with missing values:", nfl1_cleaned.shape)



Shape of DataFrame after removing columns with missing values: (15582, 22)


In [19]:
original_rows, original_cols = nfl1.shape

cleaned_rows, cleaned_cols = nfl1_cleaned.shape
lost_rows = original_rows - cleaned_rows
lost_cols = original_cols - cleaned_cols

print("Number of rows lost:", lost_rows)
print("Number of columns lost:", lost_cols)



Number of rows lost: 0
Number of columns lost: 80


We've lost quite a bit of data, but at this point we have successfully removed all the `NaN`'s from our data.

In [20]:
nfl2_cleaned = nfl2.dropna()
rows_left = nfl2_cleaned.shape[0]
print("Number of rows left after removing rows with missing values in nfl2 :", rows_left)



Number of rows left after removing rows with missing values in nfl2 : 0


In [21]:
nfl2_cleaned = nfl2.dropna(axis=1)
data_left = nfl2_cleaned.shape
print("Shape of DataFrame after removing columns with missing values in nfl2 :", data_left)


Shape of DataFrame after removing columns with missing values in nfl2 : (20262, 37)


# Filling in missing values automatically
_____

Another option is to try and fill in the missing values. For this next bit, I'm getting a small sub-section of the NFL data so that it will print well.

In [22]:
nfl_subset = nfl1.sample(n=10)
print(nfl_subset)


             Date      GameID  Drive  qtr  down   time  TimeUnder  TimeSecs  \
5635   2009-09-27  2009092700      6    2   1.0  07:19          8    2239.0   
800    2009-09-13  2009091305     12    3   NaN  07:35          8    1355.0   
2793   2009-09-20  2009092000      8    2   NaN  12:25         13    2545.0   
7935   2009-09-27  2009092712     14    4   1.0  13:59         14     839.0   
15247  2009-10-18  2009101811     21    4   NaN  11:57         12     717.0   
4371   2009-09-20  2009092010      3    1   NaN  04:35          5    2975.0   
1718   2009-09-13  2009091309      1    1   2.0  14:09         15    3549.0   
4510   2009-09-20  2009092010     22    4   1.0  03:28          4     208.0   
15146  2009-10-18  2009101811      5    2   1.0  13:24         14    2604.0   
3433   2009-09-20  2009092006      1    1   1.0  14:54         15    3594.0   

       PlayTimeDiff SideofField  ...    yacEPA  Home_WP_pre  Away_WP_pre  \
5635           31.0         CLE  ... -2.964375     0.8

We can use the Panda's fillna() function to fill in missing values in a dataframe for us. One option we have is to specify what we want the `NaN` values to be replaced with. Here, I'm saying that I would like to replace all the `NaN` values with 0.

In [23]:
nfl1_filled = nfl1.fillna(0)
print(nfl1_filled.head())


         Date      GameID  Drive  qtr  down   time  TimeUnder  TimeSecs  \
0  2009-09-10  2009091000      1    1   0.0  15:00         15    3600.0   
1  2009-09-10  2009091000      1    1   1.0  14:53         15    3593.0   
2  2009-09-10  2009091000      1    1   2.0  14:16         15    3556.0   
3  2009-09-10  2009091000      1    1   3.0  13:35         14    3515.0   
4  2009-09-10  2009091000      1    1   4.0  13:27         14    3507.0   

   PlayTimeDiff SideofField  ...    yacEPA  Home_WP_pre  Away_WP_pre  \
0           0.0         TEN  ...  0.000000     0.485675     0.514325   
1           7.0         PIT  ...  1.146076     0.546433     0.453567   
2          37.0         PIT  ...  0.000000     0.551088     0.448912   
3          41.0         PIT  ... -5.031425     0.510793     0.489207   
4           8.0         PIT  ...  0.000000     0.461217     0.538783   

   Home_WP_post  Away_WP_post  Win_Prob       WPA    airWPA    yacWPA  Season  
0      0.546433      0.453567  0.485

I could also be a bit more savvy and replace missing values with whatever value comes directly after it in the same column. (This makes a lot of sense for datasets where the observations have some sort of logical order to them.)

In [24]:
nfl1_filled = nfl1.fillna(method='ffill')
nfl1_filled = nfl1_filled.fillna(0)
print(nfl1_filled.head())



         Date      GameID  Drive  qtr  down   time  TimeUnder  TimeSecs  \
0  2009-09-10  2009091000      1    1   0.0  15:00         15    3600.0   
1  2009-09-10  2009091000      1    1   1.0  14:53         15    3593.0   
2  2009-09-10  2009091000      1    1   2.0  14:16         15    3556.0   
3  2009-09-10  2009091000      1    1   3.0  13:35         14    3515.0   
4  2009-09-10  2009091000      1    1   4.0  13:27         14    3507.0   

   PlayTimeDiff SideofField  ...    yacEPA  Home_WP_pre  Away_WP_pre  \
0           0.0         TEN  ...  0.000000     0.485675     0.514325   
1           7.0         PIT  ...  1.146076     0.546433     0.453567   
2          37.0         PIT  ...  1.146076     0.551088     0.448912   
3          41.0         PIT  ... -5.031425     0.510793     0.489207   
4           8.0         PIT  ... -5.031425     0.461217     0.538783   

   Home_WP_post  Away_WP_post  Win_Prob       WPA    airWPA    yacWPA  Season  
0      0.546433      0.453567  0.485

Filling in missing values is also known as "imputation", and you can find more exercises on it [in this lesson, also linked under the "More practice!" section](https://www.kaggle.com/dansbecker/handling-missing-values). First, however, why don't you try replacing some of the missing values in the sf_permit dataset?